In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
url = "https://academic.oup.com/ehr/search-results?f_ContentSubTypeDisplayName=Research+Article&fl_SiteID=5158&access_all=true&page=1"

In [4]:
# change to your correct driver 
driver = webdriver.Safari()

# change to your correct url
driver.get(url)
time.sleep(2)

try:
    # Click the "I'm not a robot" button
    recaptcha_button = driver.find_element(By.ID, "captcha")
    recaptcha_button.click()
    time.sleep(1)
    # Wait for the captcha to be solved (You may need to adjust the timeout)
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.ID, "userCaptchaResponse"))
    )
    time.sleep(1.1)
    # Click the "Take me to my Content" button
    submit_button = driver.find_element(By.ID, "btnSubmit")
    submit_button.click()

    # You can now access your content here
    # Add your content extraction code here
except:
    print("No captcha found")

In [5]:
elements = driver.find_elements(By.CSS_SELECTOR,"a.al-pageNumber")

# Check if there are any matching elements
if elements:
    # Get the last element (maximum page number)
    last_page_element = elements[-1]

    # Get the text content of the last element
    max_page_number = last_page_element.text

In [7]:
titles = []
dois = []

In [8]:
def get_titles_doi(html_content, titles, dois):
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all instances of the provided HTML structure
    article_boxes = soup.find_all("div", class_="sr-list al-article-box al-normal clearfix")

    # Iterate through each article box and extract data
    for article_box in article_boxes:
        # Extract the title
        title_element = article_box.find("a", class_="article-link at-sr-article-title-link")
        title = title_element.find("span").text

        # Extract the DOI
        doi_element = article_box.find("div", class_="al-citation-list").find("a")
        doi = doi_element["href"]

        titles.append(title)
        dois.append(doi)

    return titles, dois

In [9]:
titels, dois = get_titles_doi(driver.page_source, titles, dois)

for i in range(2, int(max_page_number)+1):
    url = "https://academic.oup.com/ehr/search-results?f_ContentSubTypeDisplayName=Research+Article&fl_SiteID=5158&access_all=true&page=" + str(i)
    driver.get(url)
    time.sleep(2)
    titels, dois = get_titles_doi(driver.page_source, titles, dois)

In [10]:
# Create a DataFrame
data = {
    "Title": titles,
    "DOI": dois
}
df = pd.DataFrame(data)

df

,Title,DOI
0,Catholic Intellectuals and Transnational Anti-...,https://doi.org/10.1093/ehr/cead151
1,Au Nom de la Patrie: Southern Identities and P...,https://doi.org/10.1093/ehr/cead103
2,"Royal Companies, Risk Management and Sovereign...",https://doi.org/10.1093/ehr/cead107
3,Royal Attitudes to the Atlantic Slave Trade an...,https://doi.org/10.1093/ehr/cead108
4,Espionage and the 1935 Press War in Palestine:...,https://doi.org/10.1093/ehr/cead105
...,...,...
94,The Military Orders and Holy War against Chris...,https://doi.org/10.1093/ehr/CIV.CCCCX.1
95,Why was there no Marxism in Great Britain?,https://doi.org/10.1093/ehr/XCIX.CCCXCI.297
96,The motives of the earliest crusaders and the ...,https://doi.org/10.1093/ehr/XCVIII.CCCLXXXIX.721
97,The Irish Republican Army and the development ...,https://doi.org/10.1093/ehr/XCIV.CCCLXXI.318


In [11]:
driver.close()


In [12]:
df.to_csv("titles_doi.csv", index=False)

In [13]:
df = pd.read_csv("titles_doi.csv")

In [14]:
df

,Title,DOI
0,Catholic Intellectuals and Transnational Anti-...,https://doi.org/10.1093/ehr/cead151
1,Au Nom de la Patrie: Southern Identities and P...,https://doi.org/10.1093/ehr/cead103
2,"Royal Companies, Risk Management and Sovereign...",https://doi.org/10.1093/ehr/cead107
3,Royal Attitudes to the Atlantic Slave Trade an...,https://doi.org/10.1093/ehr/cead108
4,Espionage and the 1935 Press War in Palestine:...,https://doi.org/10.1093/ehr/cead105
...,...,...
94,The Military Orders and Holy War against Chris...,https://doi.org/10.1093/ehr/CIV.CCCCX.1
95,Why was there no Marxism in Great Britain?,https://doi.org/10.1093/ehr/XCIX.CCCXCI.297
96,The motives of the earliest crusaders and the ...,https://doi.org/10.1093/ehr/XCVIII.CCCLXXXIX.721
97,The Irish Republican Army and the development ...,https://doi.org/10.1093/ehr/XCIV.CCCLXXI.318
